In [1]:
!pip install -q tensorflow

In [2]:

from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.utils import plot_model
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw
import os, random, shutil
from PIL import ImageFile
import zipfile
ImageFile.LOAD_TRUNCATED_IMAGES = True


if not os.path.exists('/content/sample_data/test/train/cats/'):
  os.makedirs('/content/sample_data/test/train/cats/')
if not os.path.exists('/content/sample_data/test/train/dogs/'):
  os.makedirs('/content/sample_data/test/train/dogs/')


zip_file = "/content/drive/MyDrive/Colab Notebooks/train.zip"
startswith_cats = "cat" # начало имени файла для кошек
startswith_dogs = "dog" # начало имени файла для собак
with zipfile.ZipFile(zip_file, 'r') as z:
  for file in z.namelist():
    file_new = file.rsplit( "/", 1 )[ 1 ]
    if not (file_new.startswith(startswith_cats) or file_new.startswith(startswith_dogs)): continue
    i = int(file_new.split('.')[1])
    if file_new.startswith(startswith_cats) and i >= 10750 and i < 10850:
      z.extract(file, 'sample_data/test/')
      os.replace('sample_data/test/train/'+file_new,'sample_data/test/train/cats/'+file_new)
    elif file_new.startswith(startswith_dogs) and i >= 10750 and i < 10850:
      z.extract(file,'sample_data/test/')
      os.replace('sample_data/test/train/'+file_new,'sample_data/test/train/dogs/'+file_new)


In [3]:
import tensorflow as ts
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import backend as K

In [4]:

img_width, img_height = 150, 150 # ширина, высота изображений

train_data_dir = 'sample_data/test/train' # директория с изображениями кошек и собак для последующего обучения модели
validation_data_dir = 'sample_data/test/validation' # директория с изображениями для тестирования модели
nb_train_samples = 2000 # количество тренировочных изображений
nb_validation_samples = 800 # количество изображений для тестирования
epochs = 30 # количество эпох (итераций)
batch_size = 16 # данные разбиваются на "пакеты" размером "batch_size"

if K.image_data_format() == 'channels_first': # для сохранения упорядочевания весов при переключении модели с одного формата данных на другой
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = ts.keras.Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape)) # добавляем сверточный 2D-слой. Поскольку это 1-ый слой, необходимо задать ключевой аргумент input_shape
# 32 - количество выходных фильтров в свертке, т.е. сколько признаков будем искать
# (3, 3) - kernel_size-высота и ширина окна 2D свертки
# Имеется 32 фильтра, каждый 3x3x3 (т. е. 27 весов), плюс 1 для смещения (т. е. всего 28 весов каждый). 32 фильтра * 28 весов = 896 параметров.
model.add(Activation('relu')) # указываем функцию активации. В данном случае f(z)=max(0,z)
model.add(MaxPooling2D(pool_size=(2, 2))) # уменьшим размер исходной матрицы (этот слой обязателен после сверточных слоев)

model.add(Conv2D(32, (3, 3))) # добавляем второй сверточный слой
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3))) # добавляем третий сверточный слой (размер фильтра увеличивается, так что сеть уже может изучать более сложные представления)
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) # слой, преобразующий 2D-данные в 1D-данные
model.add(Dense(64)) # добавляем полносвязный слой с указанием числа выходов
model.add(Activation('relu'))
model.add(Dropout(0.5)) # добавляем слой, обнуляющий в модели часть весов, чтоб избежать переобучения (0,5 означает, что он отбрасывает 50% существующих соединений)
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [14]:
model.load_weights("Model_10k_images.h5")

In [6]:
!mkdir test
!mkdir test/cat
!mkdir test/dog
!cp sample_data/test/train/cats/cat.10773.jpg test/cat/cat.jpg
!cp sample_data/test/train/dogs/dog.10773.jpg test/dog/dog.jpg

In [24]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_width, img_height = 150, 150
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    "test",
    target_size=(img_width, img_height),
    batch_size=2,
    class_mode='binary',
    shuffle=False)

imgs, labels = next(test_generator)
array_imgs=np.transpose(np.asarray([img_to_array(img) for img in imgs]),(0,2,1,3))
predictions=(model.predict(imgs)).flatten()

print(predictions)
print(1-0.12875177)
print(labels)
# если label=0 - это кот и ответ = 1 - pred


Found 2 images belonging to 2 classes.
1/1 [==============================] - 0s 63ms/step
[0.12875177 0.8880785 ]
0.87124823
[0. 1.]


In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_width, img_height = 150, 150
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    "/content/sample_data/test/train",
    target_size=(img_width, img_height),
    batch_size=200,
    class_mode='binary',
    shuffle=False)

Found 200 images belonging to 2 classes.


In [22]:
imgs, labels = next(test_generator)
array_imgs=np.transpose(np.asarray([img_to_array(img) for img in imgs]),(0,2,1,3))
predictions=np.round(model.predict(imgs)).flatten()
print(predictions)
print("okeoke")
print(labels)

7/7 [==============================] - 3s 362ms/step
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1.
 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0.
 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0.
 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1.
 1. 1. 1. 1. 1. 0. 1. 0.]
okeoke
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1

In [23]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

print("f1")
print(f1_score(labels, predictions, average='macro'))

print("precision")
print(precision_score(labels, predictions, average='macro'))

print("recall")
print(recall_score(labels, predictions, average='macro'))


print("accuracy")
print(accuracy_score(labels, predictions))

f1
0.7767857142857144
precision
0.7971137521222411
recall
0.78
accuracy
0.78
